# 🧪 Chemical Report Generator — Build & Usage Guide

This tool will let you and your lab partner quickly generate tables of chemical properties 
(density, formula, molecular mass, mp/bp, hazards) **plus structure images** directly in 
a Google Doc. Later, the Doc can be exported as a polished PDF for lab reports.

---

## 1️⃣ Setup & Requirements

### Install Python packages
```bash
pip install pubchempy google-api-python-client google-auth-httplib2 google-auth-oauthlib requests
```

### Google Cloud Setup

1. Create a project in [Google Cloud Console](https://console.cloud.google.com/).

2. Enable Google Docs API and Google Drive API.

3. Create OAuth credentials (Desktop App or Service Account).

4. Download `credentials.json` to your project folder.

5. First run → authenticate in browser → saves `token.json` for future runs.

---

## 2️⃣ Fetching Chemical Data

Use **PubChemPy** to query compound info:

- 🧬 Formula
- ⚖️ Molecular Weight  
- 🌡️ Melting / Boiling Point
- 💧 Density
- ☣️ Hazards (when available)

Augment with your own lab-specific dictionary (psychoactive compounds, stability notes, extraction solvents).

---

## 3️⃣ Getting Chemical Structure Images

Two options:

### Fast (Recommended): PubChem API

Download PNG directly:
```
https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{compound}/PNG
```

Saves time and always returns a decent structure.

### Custom: RDKit

Use RDKit to draw molecules from SMILES strings.

More control, but requires installing RDKit locally.

---

## 4️⃣ Inserting Data into Google Docs

Use the Docs API:

1. Create a table with rows for each property (formula, MM, mp/bp, density, hazards).

2. Insert chemical structure image into the last row.

3. You can format headers with bold and add emojis for readability.

**Example table layout:**

| Property | Value |
|----------|-------|
| Name | Valerenic acid |
| Formula 🧬 | C15H22O2 |
| Mol Mass ⚖️ | 234.33 g/mol |
| Density 💧 | 1.06 g/mL |
| MP/BP 🌡️ | 134–137°C / 375°C |
| Hazards ☣️ | Psychoactive, handle carefully |
| Structure 🖼 | (inserted image) |

---

## 5️⃣ Export as PDF

Once data is in the Doc:

- Use Google Drive API to export as PDF programmatically.
- Or your lab partner can simply use "File → Download as PDF" inside Docs.

---

## 6️⃣ Putting It All Together

Build a Python function like:

```python
def generate_lab_report(compounds, doc_id):
    for compound in compounds:
        data = fetch_pubchem_data(compound)
        image = fetch_pubchem_structure(compound)
        insert_table_in_doc(doc_id, data, image)
```

Script can handle multiple compounds in one run.

---

## 7️⃣ Workflow for Your Lab Partner

1. Open Google Docs template for lab reports.

2. Run the script:
   ```bash
   python lab_report.py "psilocybin" "thc" "valerenic acid"
   ```

3. Script pulls:
   - Data from PubChem + your lab dictionary.
   - Structure image (PNG).
   - Inserts a formatted table in the Doc.

4. He writes the rest of the report (methods, results, discussion).

5. Export as PDF for submission.

---

## 8️⃣ Time Estimate

- **Basic version** (text only): 1–2 hours.
- **With structures + Docs integration**: 4–6 hours.
- **Polished, auto-PDF workflow**: 1–2 days of refinement.

---

## 🚀 Extensions (Future Ideas)

- Build a **Streamlit front-end** → type compound → see table → push to Google Docs.
- Create a **Docs Add-On** (Apps Script) so he types compound names directly in the Doc, and they auto-populate.
- Add **NMR/IR spectra links** from PubChem for deeper analysis.
- Save a **personal compound database** to avoid re-fetching.

In [ ]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
import pubchempy as pcp

# 🔐 Google Docs setup
SERVICE_ACCOUNT_FILE = 'credentials.json'
SCOPES = ['https://www.googleapis.com/auth/documents']

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
docs_service = build('docs', 'v1', credentials=creds)

# 🧪 Your lab-specific info
lab_compounds = {
    'psilocybin': {
        'Category': 'Tryptamine',
        'Precursor': 'Tryptophan',
        'Best Extraction Solvent': 'Methanol/Water',
        'Stability Notes': 'Light-sensitive, store dark',
        'Bioavailability': 'Low (prodrug)',
        'Active Metabolite': 'Psilocin'
    },
    'thc': {
        'Category': 'Cannabinoid',
        'Precursor': 'THCA (decarboxylation)',
        'Best Extraction Solvent': 'Ethanol/CO2',
        'Stability Notes': 'Stable, light-sensitive',
        'Bioavailability': 'Variable',
        'Active Metabolite': '11-OH-THC'
    }
}

def get_chem_info(compound_name):
    try:
        compound = pcp.get_compounds(compound_name, 'name')[0]
        return {
            'Name': compound.iupac_name,
            'Formula': compound.molecular_formula,
            'Molar Mass': f"{compound.molecular_weight:.2f} g/mol",
            'Density': getattr(compound, 'density', 'N/A'),
            'Melting Point': getattr(compound, 'melting_point', 'N/A'),
            'Boiling Point': getattr(compound, 'boiling_point', 'N/A'),
        }
    except Exception as e:
        print(f"❌ PubChem error: {e}")
        return None

def analyze_and_push_to_docs(doc_id, compound_name):
    chem_info = get_chem_info(compound_name)
    lab_info = lab_compounds.get(compound_name.lower(), {})
    
    if not chem_info:
        return
    
    # Merge info
    data = {**chem_info, **lab_info}
    
    # Build table rows
    table_rows = [[k, str(v)] for k, v in data.items()]
    
    # Insert table into Google Doc
    requests = [{
        'insertTable': {
            'rows': len(table_rows),
            'columns': 2,
            'location': {'index': 1}
        }
    }]
    
    # Add table content
    for row_idx, row in enumerate(table_rows):
        for col_idx, cell in enumerate(row):
            requests.append({
                'insertText': {
                    'text': cell,
                    'location': {
                        'index': 1,
                        'segmentId': '',
                    }
                }
            })
    
    docs_service.documents().batchUpdate(
        documentId=doc_id, body={'requests': requests}
    ).execute()

# 🔥 Example run
doc_id = "YOUR_DOC_ID_HERE"  # From Google Docs link
analyze_and_push_to_docs(doc_id, "psilocybin")
